In [40]:
import torch
import torch.nn as nn
from torchvision import transforms
import cv2
import numpy as np
import os
from PIL import Image
from scipy.spatial.distance import cosine

In [41]:
class VGGFaceModel(nn.Module):
    def __init__(self):
        super(VGGFaceModel, self).__init__()
        self.model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg16', pretrained=True)
        self.model.classifier = nn.Sequential(*list(self.model.classifier.children())[:-1])

    def forward(self, x):
        return self.model(x)

In [42]:
vgg_face = VGGFaceModel()
vgg_face.eval()

Using cache found in /Users/adpakw/.cache/torch/hub/pytorch_vision_v0.10.0
/Users/adpakw/face_recognition_system/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/adpakw/face_recognition_system/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGGFaceModel(
  (model): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (

In [43]:
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [44]:
def get_face_embedding(image, model):
    image = preprocess(image).unsqueeze(0)  # Добавляем размер батча
    with torch.no_grad():
        embedding = model(image).numpy().flatten()
    return embedding

In [45]:
def detect_faces(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    face_images = []
    for (x, y, w, h) in faces:
        face = image[y:y+h, x:x+w]
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        face_images.append(Image.fromarray(face))

    return face_images

In [46]:
def load_known_faces(known_faces_dir, model):
    known_face_encodings = []
    known_face_names = []

    for file_name in os.listdir(known_faces_dir):
        if file_name.endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(known_faces_dir, file_name)
            detected_faces = detect_faces(image_path)

            if not detected_faces:
                print(f"No face detected in {file_name}, skipping.")
                continue

            # Предполагаем, что в изображении только одно лицо (первое найденное)
            face = detected_faces[0]
            embedding = get_face_embedding(face, model)
            known_face_encodings.append(embedding)
            known_face_names.append(os.path.splitext(file_name)[0])

    return known_face_encodings, known_face_names

In [47]:
def recognize_faces(input_image_path, known_face_encodings, known_face_names, model, threshold=0.5):
    detected_faces = detect_faces(input_image_path)

    results = []
    for face in detected_faces:
        input_embedding = get_face_embedding(face, model)

        for known_embedding, name in zip(known_face_encodings, known_face_names):
            similarity = 1 - cosine(input_embedding, known_embedding)  # Косинусное сходство
            if similarity >= threshold:
                results.append((name, similarity))

    if results:
        return max(results, key=lambda x: x[1])  # Возвращаем самое похожее лицо
    return ("Unknown", 0)

In [48]:
known_faces_dir = "../data/known_faces"
input_image_path = "../data/input_image.jpeg"

# Загрузка известных лиц
known_face_encodings, known_face_names = load_known_faces(known_faces_dir, vgg_face)

# Распознавание лица
name, similarity = recognize_faces(input_image_path, known_face_encodings, known_face_names, vgg_face)

# Результат
print(f"Распознанное лицо: {name} с уверенностью: {similarity:.2f}")

Распознанное лицо: kira с уверенностью: 0.51
